# Laboratory No. 3. Statistical Inference
# Excercise 2:
# PPCA with Gibbs Sampling
### presented by: Juan David Gil and Juan Sebastián Silva
### Date: 18 March 2016

In [6]:
% pylab inline
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
import pylab as pb
from matplotlib import cm
from scipy import stats
from scipy.stats import norm
from scipy.special import gamma as gammafun
from scipy.stats import chi
from scipy.stats import multivariate_normal
from scipy.stats import gamma

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


$$\newcommand{\inputScalar}{x}
\newcommand{\inputVector}{\mathbf{x}}
\newcommand{\inputMatrix}{\mathbf{X}}
\newcommand{\dataScalar}{y}
\newcommand{\dataVector}{\mathbf{y}}
\newcommand{\dataMatrix}{\mathbf{Y}}
\newcommand{\lengthScale}{\ell}
\newcommand{\mappingScalar}{w}
\newcommand{\mappingVector}{\mathbf{w}}
\newcommand{\mappingFunctionScalar}{f}
\newcommand{\mappingFunctionVector}{\mathbf{f}}
\newcommand{\dataStd}{\sigma}
\newcommand{\numData}{n}
\newcommand{\gaussianDist}[3]{\mathcal{N}\left(#1|#2,#3\right)}
\newcommand{\gaussianSamp}[2]{\mathcal{N}\left(#1,#2\right)}
\newcommand{\zerosVector}{\mathbf{0}}
\newcommand{\eye}{\mathbf{I}}
\newcommand{\noiseScalar}{\epsilon}
\newcommand{\noiseVector}{\mathbf{\epsilon}}
\newcommand{\noiseMatrix}{\mathbf{\Epsilon}}
\newcommand{\basisMatrix}{\mathbf{\Phi}}
\newcommand{\basisVector}{\mathbf{\phi}}
\newcommand{\basisScalar}{\phi}
\newcommand{\expSamp}[1]{\left<#1\right>}
\newcommand{\expDist}[2]{\left<#1\right>_{#2}}
\newcommand{\covarianceMatrix}{\mathbf{C}}
\newcommand{\meanVector}{\boldsymbol{\mu}}
\newcommand{\kernelScalar}{k}
\newcommand{\kernelVector}{\mathbf{\kernelScalar}}
\newcommand{\kernelMatrix}{\mathbf{K}}
\newcommand{\meanScalar}{\mu}
\newcommand{\ltwoNorm}[1]{\left\Vert #1 \right\Vert_2}$$

# Posterior distributions

With gibbs sampling the idea is to write down the full joint probability distribution of the parameters in order to have the approximation of the posterior, for the case of PPCA we can write down the posterior as:

$$p(\mathbf{X},\mathbf{Z},\mathbf{W},\pmb{\mu},\tau,\pmb{\alpha}) = p(\mathbf{X} \mid \mathbf{Z}, \mathbf{W}, \pmb{\mu}, \tau) p(\mathbf{Z}) p(\mathbf{W} \mid \pmb{\alpha}) p(\pmb{\mu}) p(\pmb{\alpha}) p(\tau)$$

Where:

$$p(\mathbf{X} \mid \mathbf{Z}, \mathbf{W}, \pmb{\mu}, \tau) p(\mathbf{Z})= \prod_{n=1}^{N} p(\mathbf{x_n} \mid \mathbf{z_n},\mathbf{W},\pmb{\mu},\tau) p(\mathbf{z_n}) $$ 

$$ p(\mathbf{x_n} \mid \mathbf{z_n},\mathbf{W},\pmb{\mu},\tau) = \mathcal{N}(\mathbf{W}\mathbf{z_n} + \pmb{\mu}, \tau^{-1}\mathbf{I})$$

$$ p(\mathbf{z_n}) = \mathcal{N}(\mathbf{z_n} \mid \mathbf{0},\mathbf{I})$$

$$ p(\mathbf{W} \mid \pmb{\alpha}) = \prod_{i=1}^{M} \mathcal{N}(\mathbf{w_i} \mid \mathbf{0}, \alpha_i^{-1}\mathbf{I})  $$

$$ p(\pmb{\mu}) =  \mathcal{N}(\pmb{\mu} \mid \mathbf{0}, \beta^{-1}\mathbf{I}) $$

$$ p(\pmb{\alpha}) = \prod_{i=1}^{M} Gamma(\alpha_i \mid a_\alpha, b_\alpha)$$

$$ p(\tau) = Gamma(\tau \mid a_\tau, b_\tau)$$

The the posteriors can be derived as follows:

### Posterior for the latent space variable $\mathbf{z_n}$

The posterior for X it's not necccesary because it's the data itself, then one may start to derive the posterior for latent space variable($\mathbf{z_n}$) conditioned to the other variables, the form of the posterior it's as follows :

$$ p(\mathbf{z_n} \mid \mathbf{X}, \mathbf{z_1},...,\mathbf{z_{n-1}},\mathbf{z_{n+1}},...,\mathbf{z_N},\mathbf{W}, \pmb{\alpha}, \tau, \pmb{\mu}) = \mathcal{N}(\mathbf{z_n} \mid \mathbf{0},\mathbf{I}) \mathcal{N}(\mathbf{W}\mathbf{z_n} + \pmb{\mu}, \tau^{-1}\mathbf{I}) $$

taking $ log(p(\mathbf{z_n} \mid ...))$, and after several derivations one can obtain again a normal distribution representing the posterior for $\mathbf{z_n}$:

$$ log(p(\mathbf{z_n} \mid ...)) = Constant - (\frac{1}{2}\mathbf{z_n^T}\mathbf{z_n}) - (\frac{1}{2}(\mathbf{x_n} - \mathbf{C})^T \tau\mathbf{I} (\mathbf{x_n} - \mathbf{C}))$$

where $\mathbf{C} = \mathbf{W}\mathbf{z_m} + \pmb{\mu}$

The multiplication of a normal distribution with another normal distribution arises another normal dsitribution, then one need to get the similar cuadratic form of the normal distribution, we btain the following expression from the one above:

$$ log(p(\mathbf{z_n} \mid ...)) = Constant - \frac{1}{2}[\mathbf{z_n^T}(\mathbf{D})\mathbf{z_n} - 2\tau\mathbf{z_n^T}\mathbf{W}^T(\mathbf{x_n} - \pmb{\mu}) + Constant]$$

where $ \mathbf{D_{zn}} = (\mathbf{I} + \tau(\mathbf{W}^T\mathbf{W})) $. As one may realize, the cuadratic form for the normal distribution is already there, then the posterior is a Multivariate Gaussian as follows:

$$ p(\mathbf{z_n} \mid ...) = \mathcal{N}(\mathbf{z_n} \mid \pmb{\mu_{z_n}}, \mathbf{D_{zn}}^{-1}), $$

$$ \pmb{\mu_{z_n}}  = \mathbf{D_{zn}}^{-1}\tau\mathbf{W}^T(\mathbf{x_n} - \pmb{\mu})$$

### Posterior for the projection vectors $\mathbf{w_i}$

In order to make easire computations, one can avoid to sample from distribution for matrices as $ p(\mathbf{W} \mid \pmb{\alpha})$ but for each one of the column vectors $\mathbf{w_i}$ from $\mathbf{W}$ instead, then the posterior distribution  can be derived from the joint as:

$$ p(\mathbf{w_i} \mid \mathbf{X}, \mathbf{Z}, \mathbf{w_1},...,\mathbf{w_{i-1}},\mathbf{w_{i+1}},...,\mathbf{w_M}, \pmb{\alpha}, \tau, \pmb{\mu}) = \prod_{n=1}^{N} \mathcal{N}(\mathbf{W}\mathbf{z_n} + \pmb{\mu}, \tau^{-1}\mathbf{I})\mathcal{N}(\mathbf{w_i} \mid \mathbf{0}, \alpha_i^{-1}\mathbf{I})  $$

Like in the case showed before, one may obtain a normal distribution aranging the terms in such a way that can compare the cuadratic form with the one of the multivariate normal. Taking $ log(p(\mathbf{w_i} \mid ...))$:

$$ log(p(\mathbf{w_i} \mid ...)) = Constant - \frac{1}{2}[\sum_{n=1}^{N}(\mathbf{x_n}^T - (\mathbf{w_i z_{in}} + \mathbf{A})^T)(\tau^{-1}\mathbf{I})^{-1}(\mathbf{x_n} - \mathbf{w_i z_{in}} + \mathbf{A})) + \mathbf{w_i}^T(\alpha_i^{-1}\mathbf{I})^{-1}\mathbf{w_i}]$$

Where $A = \mathbf{W_{/i}}\mathbf{z_n} + \pmb{\mu}$ and $\mathbf{W_{/i}}$ means thaht the ith column from $\mathbf{W}$ is denied, the reason for this is that only the ith column is the variable of interest to the posterior. After some other derivations one obtains the following cuadratic form:

$$ log(p(\mathbf{w_i} \mid ...)) = Constant - \frac{1}{2}[\mathbf{w_i}^T(\alpha_i^{-1}\mathbf{I} + \tau b\mathbf{I})^{-1}\mathbf{w_i} -2\tau\mathbf{w_i}^T(\sum_{n=1}^{N}z_{in}\mathbf{x_n} - z_{in}\mathbf{A})]$$

where $b = \sum_{n=1}^{N}z_{in}^2$, here $z_{in}$ makes reference to the ith input from the nth latent space vector representation. In the expresion above, again, one can identify the terms to construct a multivariate normal, then the posterior distribution can be defined as follows:

$$ p(\mathbf{z_n} \mid ...) = \mathcal{N}(\mathbf{w_i} \mid \pmb{\mu_{w_i}}, \mathbf{D_{wi}}^{-1}), $$

Where:

$$\mathbf{D_{wi}} = (\alpha_i^{-1}\mathbf{I} + \tau b\mathbf{I})^{-1},$$

$$\pmb{\mu_{w_i}} = \mathbf{D_{wi}}^{-1}\tau(\sum_{n=1}^{N}z_{in}\mathbf{x_n} - z_{in}\mathbf{A})$$

### Posterior for $\pmb{\alpha_i}$

This are the parameters which controls the distribution of $\mathbf{W}$ we can write down the posterior distribution for $\pmb{\alpha_i}$ as follows:

$$ p(\mathbf{\alpha_i} \mid \mathbf{X}, \mathbf{Z}, \mathbf{W}, \alpha_1,...,\alpha_{i-1},\alpha_{i+1},...,\alpha_M, \tau, \pmb{\mu}) = \mathcal{N}(\mathbf{w_i} \mid \mathbf{0}, \alpha_i^{-1}\mathbf{I}) Gamma(\alpha_i \mid a_\alpha, b_\alpha)$$

In this case the calculations will be done with the posterior directly, not with log of the posterior, this is due to simplicity. The idea here is to obtain a posterior that is going to be a Gamma distribution also, then writing the expressions for each distribution one obtains the following:

$$ p(\alpha_i \mid ...) = Const * \alpha_i^{D/2} \alpha_i^{a_{\alpha} - 1} exp\Big\{ -\frac{1}{2}\mathbf{w_i}^T(\alpha_i^{-1}\mathbf{I})^{-1}\mathbf{w_i} - b_{\alpha}\alpha_i \Big\}$$

AS our variable of interest is $\alpha_i$ we factorize in the expresion above and after some calculations we have the exponential form of a Gamma distribution as follows:

$$ p(\alpha_i \mid ...) = Const * \alpha_i^{(D/2 + a_{\alpha} - 1)} exp\Big\{ -(\frac{1}{2}\mathbf{w_i}^T\mathbf{w_i} + b_{\alpha})\alpha_i \Big\}$$

Then the Gamma distribution can be specified in the next form:

$$ p(\alpha_i \mid ...) = Gamma(\alpha_i \mid a_{\alpha post}, b_{\alpha post}),$$

$$ a_{\alpha post} =  D/2 + a_{\alpha}, $$

$$b_{\alpha post} = \frac{1}{2}\mathbf{w_i}^T\mathbf{w_i} + b_{\alpha} $$

### Posterior for $\tau$

The posterior for $\tau$ cand be expresed as follows:

$$ p(\tau \mid \mathbf{X}, \mathbf{Z}, \mathbf{W}, \pmb{\alpha},\pmb{\mu}) = \prod_{n=1}^{N} \mathcal{N}(\mathbf{W}\mathbf{z_n} + \pmb{\mu}, \tau^{-1}\mathbf{I}) Gamma(\tau \mid a_\tau, b_\tau)$$

Making a similar derivation like the one before one can getn expresion similar to the gamma distribution, in this case it's as follows:

$$ p(\tau \mid ...) = Const * \tau^{((ND/2) + a_{\tau} - 1)}exp\Big\{ -(\frac{1}{2}\sum_{n=1}^{N}(\mathbf{x_n}- \mathbf{B})^T(\mathbf{x_n}- \mathbf{B}) + b_{\tau})\tau\Big\} $$

Where $\mathbf{B} = \mathbf{Wz_n} + \pmb{\mu}$Then we can write the posterior for $\tau$ as:

$$ p(\tau \mid ...) = Gamma(\tau \mid a_{\tau post}, b_{\tau post}),$$

$$ a_{\tau post} = (ND/2) + a_{\tau}, $$

$$b_{\tau post} = \frac{1}{2}\sum_{n=1}^{N}(\mathbf{x_n}- \mathbf{B})^T(\mathbf{x_n}- \mathbf{B}) + b_{\tau}$$

### Posterior for $\pmb{\mu}$

The posterior for $\pmb{\mu}$ is also a Gaussian, let's take a look to the expression and make the derivations to obtain that posterior, this can be write down as follows:

$$ p(\pmb{\mu} \mid \mathbf{X}, \mathbf{Z}, \mathbf{W}, \pmb{\alpha},\tau) = \prod_{n=1}^{N} \mathcal{N}(\mathbf{W}\mathbf{z_n} + \pmb{\mu}, \tau^{-1}\mathbf{I}) \mathcal{N}(\pmb{\mu} \mid \mathbf{0}, \beta^{-1}\mathbf{I}) $$

For this case the log of the posterior was not taken, then one can write the posterior in the following way:

$$ p(\pmb{\mu} \mid ...) = Constant * exp\Big\{ -\frac{1}{2}(\sum_{n=1}^{N}(\mathbf{x_n}- \mathbf{B})^T(\mathbf{x_n}- \mathbf{B})\tau - \pmb{\mu}^T\pmb{\mu}\beta )\Big\}  $$

$ \mathbf{B}$ is already defined for the previous posterior, making some other derivations one can get to the cuadratic form on the gaussian as can be shown below:

$$ p(\pmb{\mu} \mid ...) = Constant * exp\Big\{ -\frac{1}{2}[\pmb{\mu}^T(\beta + N\tau)\pmb{\mu} - 2\pmb{\mu}^T\tau(\sum_{n=1}^{N}\mathbf{x_n} - \mathbf{Wz_n})]\Big\}  $$

Then the posterior for $\pmb{\mu}$ that is a multiavriate Normal is defined as follows:

$$ p(\pmb{\mu} \mid ...) = \mathcal{N}(\pmb{\mu} \mid \pmb{\mu_{post}}, \pmb{\Sigma_{\mu post}}),$$

$$ \pmb{\Sigma_{\mu post}} = (\beta\mathbf{I} + N\tau\mathbf{I})^{-1} $$

$$ \pmb{\mu_{post}} = \pmb{\Sigma_{\mu post}}(\tau(\sum_{n=1}^{N}\mathbf{x_n} - \mathbf{Wz_n})) $$

# Gibbs sampling for PPCA